## First wrap existing property lists into a dict structure and output as yaml

In [1]:
from camproperties import scmos_noise_properties, emCCD_properties
import yaml

In [2]:
scmos_noise_properties.keys()

['VSC-00954', 'CSC-00425', 'VSC-02698', 'VSC-02858']

In [3]:
scmos_noise_properties['VSC-00954']

{'12-bit (high well capacity)': {'ADOffset': 100,
  'ElectronsPerCount': 6.97,
  'ReadNoise': 5.96,
  'SaturationThreshold': 2047},
 '12-bit (low noise)': {'ADOffset': 100,
  'ElectronsPerCount': 0.28,
  'ReadNoise': 1.1,
  'SaturationThreshold': 2047},
 '16-bit (low noise & high well capacity)': {'ADOffset': 100,
  'ElectronsPerCount': 0.5,
  'ReadNoise': 1.33,
  'SaturationThreshold': 65535}}

In [4]:
def wrapNPsCMOS(cdict):
    configdict = {}
    for serial in cdict:
        camdict = {}
        camdict['properties'] = {'noiseProperties': cdict[serial] }
        camdict['name'] = 'Andor sCMOS'
        if serial.startswith('VSC'):   
            camdict['model'] = 'Zyla'
        elif serial.startswith('CSC'):
            camdict['model'] = 'Sona'
        else:
            camdict['model'] = 'Unknown'
        configdict[serial] = camdict
        
    return configdict

In [5]:
scmos_dict = wrapNPsCMOS(scmos_noise_properties)
scmos_dict['CSC-00425']['comment'] = 'a Sona entry where the noise property names are unicode strings; not sure if this matters'

In [6]:
def getpreamp(padict,serial):
    if serial in padict:
        return padict[serial]
    else:
        return 2

def wrapNPsEMCCD(npdict,padict):
    configdict = {}
    for serial in npdict:
        camdict = {}
        camdict['properties'] = {'noiseProperties': npdict[serial], 'preampGain':getpreamp(padict,serial)}
        camdict['name'] = 'Andor IXON'
        camdict['model'] = 'unknown' # could these be different IXON models?
        configdict[serial] = camdict
    return configdict

In [7]:
emccd_dict = wrapNPsEMCCD(emCCD_properties['noiseProperties'],emCCD_properties['preampGain'])

In [8]:
# here we dump these dicts into yamls for the two camera types separately
yaml.safe_dump(scmos_dict, file('AndorsCMOScams.yaml','w'), encoding='utf-8', allow_unicode=True)
yaml.safe_dump(emccd_dict, file('IXONcams.yaml','w'), encoding='utf-8', allow_unicode=True)

### Heterogeneous dictionary structure that is adopted

- each camera has an entry (dict-like) accessed by its serial number
- the interesting stuff is in the `properties` slot
- the `properties` slot has, based an camera type, different sub entries
- by convention `name` and `model` slots are present to help identify cam type for the reader
- other slots can be added, e.g. a `comment` slot if so desired, for now they are completely ignored

Example of sCMOS type entry:

In [14]:
print(yaml.safe_dump( scmos_dict['CSC-00425'], encoding='utf-8', allow_unicode=True, default_flow_style=False, default_style=''))

comment: a Sona entry where the noise property names are unicode strings; not sure
  if this matters
model: Sona
name: Andor sCMOS
properties:
  noiseProperties:
    12-bit (low noise):
      ADOffset: 100
      ElectronsPerCount: 0.45
      ReadNoise: 1.21
      SaturationThreshold: 1776
    16-bit (high dynamic range):
      ADOffset: 100
      ElectronsPerCount: 1.08
      ReadNoise: 1.84
      SaturationThreshold: 44185

